In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


## This week’s topic: Oil prices

But then, even bigger news hit, at least for data nerds: Pandas 2.0 was released. Among the biggest changes in Pandas 2.0 is the use of PyArrow for in-memory storage of values. NumPy is still the default, and it’s not necessarily going away, but PyArrow is the clear direction for the future of Pandas.

### Data

We’ll look at oil prices and production statistics from the OECD, a group of countries that the Economist calls “a club of mostly rich countries.” (Maybe it’s just me, but I always giggle when I read that description.) In addition to using Pandas, I’ll be taking advantage of the %timeit and %%timeit magic commands in Jupyter to compare performance. You’re welcome to check the timing in other ways, if you prefer.

Our data set this week will come from the OECD’s oil import price page, at https://data.oecd.org/energy/crude-oil-import-prices.htm. We want the years 1980-2021 for all countries. Download the "full indicator data" as a CSV file, and save it. Note that the filename will reflect the date and time at which your download takes place, so my filename and yours will differ at the end.

Each of the questions should be performed twice, using `%timeit` to check how long each one took:

1. With the default engine and data backend
1. With the PyArrow engine and PyArrow backend

### Tasks

This week’s questions:

1. Load the data into a data frame. We only want to load five columns: LOCATION, FREQUENCY, and TIME, Value, and Flag Codes. (Note the odd capitalization.)
2. Check the memory usage of the data frame. (You don’t need to time this.)
3. Keep only those rows with a monthly frequency (i.e., M).
4. Create two new integer columns, YEAR and MONTH, based on the existing “TIME” column.
5. What has been the per-country import price, taken over all measurements? Which countries have paid above the mean?
6. Grouping by YEAR then MONTH, find the mean oil-import price across all countries. When was the mean price the highest? The lowest?
7. When you’re done, produce a table (a data frame, maybe?) indicating whether PyArrow was faster or slower, and by how much.

This week’s learning goals include working with PyArrow, checking memory usage, and using “%timeit” to time how quickly your code runs.

#### 1. Load the data into a data frame. We only want to load five columns: LOCATION, FREQUENCY, and TIME, Value, and Flag Codes. (Note the odd capitalization.)

In [8]:
data = "data/DP_LIVE_31012024165916770.csv"

In [11]:
df = pd.read_csv(
    data,
    usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"]
)

In [13]:
%%timeit
df = pd.read_csv(
    data, 
    usecols=['LOCATION', 'FREQUENCY', 'TIME', 'Value', 'Flag Codes']
)

19.1 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
df.shape

(21547, 5)

In [16]:
df = pd.read_csv(
    data,
    usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"],
    engine="pyarrow",
    dtype_backend="pyarrow",
)

In [17]:
%%timeit
df = pd.read_csv(
    data, 
    usecols=['LOCATION', 'FREQUENCY', 'TIME', 'Value', 'Flag Codes'],
    engine='pyarrow', 
    dtype_backend='pyarrow'
)

15.4 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 2. Check the memory usage of the data frame. (You don’t need to time this.)

In [ ]:
df = pd.read_csv(data, usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"])

In [15]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21547 entries, 0 to 21546
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LOCATION    21547 non-null  object 
 1   FREQUENCY   21547 non-null  object 
 2   TIME        21547 non-null  object 
 3   Value       11504 non-null  float64
 4   Flag Codes  10043 non-null  object 
dtypes: float64(1), object(4)
memory usage: 4.8 MB


In [ ]:
df = pd.read_csv(
    data,
    usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"],
    engine="pyarrow",
    dtype_backend="pyarrow",
)

In [20]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21547 entries, 0 to 21546
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   LOCATION    21547 non-null  string[pyarrow]
 1   FREQUENCY   21547 non-null  string[pyarrow]
 2   TIME        21547 non-null  string[pyarrow]
 3   Value       11504 non-null  double[pyarrow]
 4   Flag Codes  10043 non-null  string[pyarrow]
dtypes: double[pyarrow](1), string[pyarrow](4)
memory usage: 748.0 KB


#### 3. Keep only those rows with a monthly frequency (i.e., M).

In [30]:
df = pd.read_csv(data, usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"])
df

,LOCATION,FREQUENCY,TIME,Value,Flag Codes
0,AUS,A,1980,31.81,NaN
1,AUS,A,1981,35.88,NaN
2,AUS,A,1982,35.42,NaN
3,AUS,A,1983,30.88,NaN
4,AUS,A,1984,29.19,NaN
...,...,...,...,...,...
21542,LTU,M,2023-05,NaN,C
21543,LTU,M,2023-06,NaN,C
21544,LTU,M,2023-07,NaN,C
21545,LTU,M,2023-08,NaN,C


In [24]:
df[df["FREQUENCY"] == "M"]

,LOCATION,FREQUENCY,TIME,Value,Flag Codes
5668,AUS,M,1980-01,27.81,NaN
5669,AUS,M,1980-02,29.81,NaN
5670,AUS,M,1980-03,29.90,NaN
5671,AUS,M,1980-04,31.30,NaN
5672,AUS,M,1980-05,31.24,NaN
...,...,...,...,...,...
21542,LTU,M,2023-05,NaN,C
21543,LTU,M,2023-06,NaN,C
21544,LTU,M,2023-07,NaN,C
21545,LTU,M,2023-08,NaN,C


In [31]:
%%timeit
df[df["FREQUENCY"] == 'M']

1.87 ms ± 82.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [32]:
%%timeit
df[(df["FREQUENCY"] == "M") & (df["LOCATION"] == "AUS") & (df["Value"] > 30)]

1.93 ms ± 82.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [33]:
df = pd.read_csv(
    data,
    usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"],
    engine="pyarrow",
    dtype_backend="pyarrow",
)

In [34]:
%%timeit
df[df["FREQUENCY"] == 'M']

1.18 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [35]:
%%timeit
df[(df["FREQUENCY"] == "M") & (df["LOCATION"] == "AUS") & (df["Value"] > 30)]

692 µs ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### 4. Create two new integer columns, YEAR and MONTH, based on the existing “TIME” column.

In [93]:
df = pd.read_csv(data, usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"])
df = df[df["FREQUENCY"] == "M"]
df

,LOCATION,FREQUENCY,TIME,Value,Flag Codes
5668,AUS,M,1980-01,27.81,NaN
5669,AUS,M,1980-02,29.81,NaN
5670,AUS,M,1980-03,29.90,NaN
5671,AUS,M,1980-04,31.30,NaN
5672,AUS,M,1980-05,31.24,NaN
...,...,...,...,...,...
21542,LTU,M,2023-05,NaN,C
21543,LTU,M,2023-06,NaN,C
21544,LTU,M,2023-07,NaN,C
21545,LTU,M,2023-08,NaN,C


In [94]:
df["YEAR"] = df["TIME"].str.slice(0, 4).astype("int64")
df["MONTH"] = df["TIME"].str.slice(5, 7).astype("int64")
df[["YEAR", "MONTH"]]

,YEAR,MONTH
5668,1980,1
5669,1980,2
5670,1980,3
5671,1980,4
5672,1980,5
...,...,...
21542,2023,5
21543,2023,6
21544,2023,7
21545,2023,8


In [95]:
%%timeit
df["TIME"].str.slice(0, 4).astype("int64")
df["TIME"].str.slice(5, 7).astype("int64")

5.79 ms ± 311 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [96]:
df = pd.read_csv(
    data,
    usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"],
    engine="pyarrow",
    dtype_backend="pyarrow",
)
df = df[df["FREQUENCY"] == "M"]

In [97]:
%%timeit
df["TIME"].str.slice(0, 4).astype("int64[pyarrow]")
df["TIME"].str.slice(5, 7).astype("int64[pyarrow]")

695 µs ± 31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### 5. What has been the per-country import price, taken over all measurements? Which countries have paid above the mean?

In [70]:
df = pd.read_csv(data, usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"])
df = df[df["FREQUENCY"] == "M"]
df

,LOCATION,FREQUENCY,TIME,Value,Flag Codes
5668,AUS,M,1980-01,27.81,NaN
5669,AUS,M,1980-02,29.81,NaN
5670,AUS,M,1980-03,29.90,NaN
5671,AUS,M,1980-04,31.30,NaN
5672,AUS,M,1980-05,31.24,NaN
...,...,...,...,...,...
21542,LTU,M,2023-05,NaN,C
21543,LTU,M,2023-06,NaN,C
21544,LTU,M,2023-07,NaN,C
21545,LTU,M,2023-08,NaN,C


In [ ]:
country_means = df.groupby("LOCATION")["Value"].mean()

In [ ]:
country_means.loc[country_means > country_means.mean()]

In [71]:
%%timeit 
country_means = df.groupby("LOCATION")["Value"].mean()
country_means.loc[country_means > country_means.mean()]

1.04 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [72]:
df = pd.read_csv(
    data,
    usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"],
    engine="pyarrow",
    dtype_backend="pyarrow",
)
df = df[df["FREQUENCY"] == "M"]

In [73]:
%%timeit 
country_means = df.groupby("LOCATION")["Value"].mean()
country_means.loc[country_means > country_means.mean()]

958 µs ± 79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### 6. Grouping by YEAR then MONTH, find the mean oil-import price across all countries. When was the mean price the highest? The lowest?

In [74]:
df = pd.read_csv(data, usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"])
df = df[df["FREQUENCY"] == "M"]
df["YEAR"] = df["TIME"].str.slice(0, 4).astype("int64")
df["MONTH"] = df["TIME"].str.slice(5, 7).astype("int64")
df[["YEAR", "MONTH"]]

,YEAR,MONTH
5668,1980,1
5669,1980,2
5670,1980,3
5671,1980,4
5672,1980,5
...,...,...
21542,2023,5
21543,2023,6
21544,2023,7
21545,2023,8


In [89]:
df.groupby(["LOCATION", "YEAR", "MONTH"])["Value"].mean().groupby("LOCATION").agg(["idxmin", "min", "idxmax", "max"])

,idxmin,min,idxmax,max
LOCATION,,,,
AUS,"(AUS, 1986, 8)",10.13,"(AUS, 2008, 7)",142.29
AUT,"(AUT, 1986, 7)",11.02,"(AUT, 1981, 9)",39.72
BEL,"(BEL, 1986, 7)",8.59,"(BEL, 2008, 6)",127.28
CAN,"(CAN, 1998, 12)",10.68,"(CAN, 2008, 7)",133.99
CHE,"(CHE, 1988, 10)",12.59,"(CHE, 1988, 6)",15.40
CZE,"(CZE, 2003, 11)",27.71,"(CZE, 2008, 7)",131.20
DEU,"(DEU, 1998, 12)",9.70,"(DEU, 2008, 6)",131.81
DNK,"(DNK, 1986, 7)",10.28,"(DNK, 2022, 6)",127.51
ESP,"(ESP, 1986, 7)",9.28,"(ESP, 2008, 7)",130.14


In [90]:
%%timeit
df.groupby(["LOCATION", "YEAR", "MONTH"])["Value"].mean().groupby("LOCATION").agg(["idxmin", "min", "idxmax", "max"])

6.71 ms ± 612 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [91]:
df = pd.read_csv(
    data,
    usecols=["LOCATION", "FREQUENCY", "TIME", "Value", "Flag Codes"],
    engine="pyarrow",
    dtype_backend="pyarrow",
)
df = df[df["FREQUENCY"] == "M"]
df["YEAR"] = df["TIME"].str.slice(0, 4).astype("int64")
df["MONTH"] = df["TIME"].str.slice(5, 7).astype("int64")
df[["YEAR", "MONTH"]]

,YEAR,MONTH
5668,1980,1
5669,1980,2
5670,1980,3
5671,1980,4
5672,1980,5
...,...,...
21542,2023,5
21543,2023,6
21544,2023,7
21545,2023,8


In [92]:
%%timeit
df.groupby(["LOCATION", "YEAR", "MONTH"])["Value"].mean().groupby("LOCATION").agg(["idxmin", "min", "idxmax", "max"])

8.56 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
